### Segmenting and Clustering Neighborhoods in Toronto

First, let's scrape the wikipedia page at the address https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M and obtain the data that is in the table of postal codes

In [1]:
#Import libraries

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Request page
response = requests.get(url)


In [3]:
#Do the soup, find table, isolate rows
soup = BeautifulSoup(response.text)
table = soup.find('tbody')
tablerows = table.find_all('tr')[1:]

In [4]:
#Define df columns 
cols = ['PostCode', 'Borough', 'Neighbourhood'] 
#Initialize df
df = pd.DataFrame(columns=cols)

In [5]:
#Add rows to df
for i in range(len(tablerows)):
    df.loc[i] = [td.text.strip() for td in tablerows[i].find_all('td')]

In [6]:
#Check
df.head()

,PostCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Polish the df according to the following requirements:
    
* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [7]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df.drop(df[df.Borough == 'Not assigned'].index)

In [8]:
df.head(10)

,PostCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [9]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
#So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns 
#will be Queen's Park.

df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [10]:
#More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, 
#you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows 
#will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

df = df.groupby(['PostCode', 'Borough'])['Neighbourhood'].apply(list).to_frame().reset_index()

#Remove square brackets
df['Neighbourhood'] = df['Neighbourhood'].str.join(', ')

In [13]:
#Final result
df.shape

(103, 3)